# Import

In [7]:
import json
import pathlib
import sys

import numpy
from tqdm import tqdm

sys.path.append(pathlib.Path.cwd().parent.as_posix())
from src.data.preprocess.lib.utils import filtered_patient_number_zfill_range

# Path

In [8]:
project_root = pathlib.Path.cwd().parent
gated_root = list(project_root.rglob("Gated*"))[0]

# Finding class distribution

In [17]:
total_dcm = 0
# patient_list = filtered_patient_number_zfill_range(0,789)
split_dict = get_patient_split([0.7, 0.2, 0.1])

for patient_list in split_dict.values():
    for patient in tqdm(patient_list):
        if patient != "000":
            lstrip_patient = patient.lstrip("0")
        else:
            lstrip_patient = "0"
        patient_root = list(gated_root.rglob(lstrip_patient))[0]
        total_dcm += len(list(patient_root.rglob("*.dcm")))


100%|███████████████████████████████████████████████| 76/76 [00:05<00:00, 13.44it/s]


In [18]:
total_pixel = total_dcm*512*512
print(f"{total_dcm=}")    
print(f"{total_pixel=}")

total_dcm=36731
total_pixel=9628811264


In [19]:
clean_json_path = list(project_root.rglob("clean*.json"))[0]
segmentation_dict = {}
with clean_json_path.open(mode="r") as f:
    clean_dict = json.load(f)
    for images in tqdm(clean_dict.values()):
        for image in images:
            for lesion in image["roi"]:
                label = lesion["loc"]
                n_pos = len(lesion["loc"])
                segmentation_dict[label] = segmentation_dict.get(label,0) + n_pos
background_pixel = total_pixel
for segmentation in segmentation_dict.values():
    background_pixel -=segmentation

100%|██████████████████████████████████████████| 417/417 [00:00<00:00, 82129.26it/s]


In [20]:
bg_percent = background_pixel/total_pixel*100
rca_percent = segmentation_dict["RCA"]/total_pixel*100
lad_percent = segmentation_dict["LAD"]/total_pixel*100
lcx_percent = segmentation_dict["LCX"]/total_pixel*100
lca_percent = segmentation_dict["LCA"]/total_pixel*100

In [21]:
print(f"{bg_percent=}")
print(f"{rca_percent=}")
print(f"{lad_percent=}")
print(f"{lcx_percent=}")
print(f"{lca_percent=}")

bg_percent=99.9998261467637
rca_percent=6.237530091024952e-05
lad_percent=6.542863731844355e-05
lcx_percent=3.819786159638657e-05
lca_percent=7.851436478213226e-06
